In [1]:
# 基幹システムのデータをアップロードするスクリプト

In [54]:
########################
# 
# 設定（変動）
# 
########################

# 該当月
applicable_month = 7

In [48]:
########################
# 
# モジュール
# 
########################

# 一般モジュール
import pandas as pd
import re
import pytz
from datetime import datetime as dt

import os
import csv
import time
from selenium.webdriver.common.by import By

# スプシ関連
from gspread_dataframe import set_with_dataframe

# 自作モジュール
import func
import conf

print("モジュールのインポート完了")

モジュールのインポート完了


In [56]:
########################
# 
# 設定（固定）
# 
########################
# 基幹システムのURL
url_kikan = "https://mensclear.com/users/reservations"

# ログイン情報
login_email = "cb140000@yahoo.co.jp" # メールアドレス
login_pass  = "muramura" # パスワード

# CSVファイルの保存先
csv_filepass = "data/reservations.csv"

# 実行月
this_month = dt.today().month

In [ ]:
########################
# 
# 動作
# 
########################

In [57]:
#########################
# 月をまたいだら実行しない
#########################
if(this_month == applicable_month):

    # クローム軌道
    drive = func.start_chrome(headless=False)

    #########################
    # 基幹システムにログイン
    #########################

    drive.get(url_kikan)
    time.sleep(3)

    # メールに入力
    elm_email = drive.find_element(By.ID, "user_email")
    elm_email.send_keys(login_email)
    time.sleep(1)

    # パスワードを入力
    elm_pass = drive.find_element(By.ID, "user_password")
    elm_pass.send_keys(login_pass)
    time.sleep(1)

    # ログインボタン押下
    elm_login_btn = drive.find_element(By.CLASS_NAME, "p-sessions__button")
    elm_login_btn.click()
    time.sleep(3)

    #########################
    # CSVダウンロード
    #########################

    # データ一覧を表示
    elm_data_index = drive.find_elements(By.CLASS_NAME, "l-dashboard__panel")[0]
    elm_data_index.click()
    time.sleep(3)

    # ダウンロードボタンをクリック
    elm_dl_btn = drive.find_element(By.NAME, "commit")
    elm_dl_btn.click()
    time.sleep(3)

    #########################
    # スプシにアップロード
    #########################

    # クラン ROAS管理シートキー
    sheet_key = "18s2UpmJ5z0EyMxepWVOZ7gDnQ8y5SOBF9Sma3P8Enws"

    # シートにアクセス
    wb = func.connect_gspread(sheet_key)

    # CSVをアップロード
    wb.values_update(
        '基幹_当月',
        params={'valueInputOption': 'USER_ENTERED'},
        body={'values': list(csv.reader(open(csv_filepass, encoding='cp932')))}
    )
    time.sleep(3)

    #########################
    # CSVファイルを削除
    #########################

    os.remove(csv_filepass)